In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchWindowException
import time
import json

# Set up the Chrome webdriver (you need to download chromedriver and specify its path)
driver = webdriver.Chrome()

def generate_stock_link(stock_logo):
    base_url = 'https://www.screener.in/company/'
    return base_url + stock_logo.upper() + '/consolidated'

def get_profit_and_loss_details(driver):
    data = {}
    # Find the profit and loss
    profit_and_loss = driver.find_element(By.ID, 'profit-loss')

    # Find the tables inside the section
    tables = profit_and_loss.find_elements(By.CLASS_NAME, 'ranges-table')

    for table in tables:
        # Extract the table title
        table_title = table.find_element(By.TAG_NAME, 'th').text.strip()
        
        # Initialize an empty dictionary to store data for this table
        table_data = {}
        
        # Find all rows in the table body
        rows = table.find_elements(By.TAG_NAME, 'tr')
    
        # Iterate over each row (skip the first row which contains the title)
        for row in rows[1:]:
            # Extract the cells in this row
            cells = row.find_elements(By.TAG_NAME, 'td')
            
            # Extract the time period (e.g., "10 Years") and the value (e.g., "66%")
            time_period = cells[0].text.strip()
            value = cells[1].text.strip()
            
            # Store the value in the table data dictionary
            table_data[time_period] = value
            
        # Store the table data in the main data dictionary with the table title as key
        data[table_title] = table_data
    return data

def get_revenue_and_net_profit_details(driver, section_class):
    # Find the section
    section = driver.find_element(By.ID, section_class)

    # Find the content div inside the section
    content_div_section = section.find_element(By.CLASS_NAME, 'fill-card-width')

    # Find the table inside the content div
    table_section = content_div_section.find_element(By.CLASS_NAME, 'data-table')

    # Find the table header (thead) element
    thead = table_section.find_element(By.TAG_NAME, 'thead')

    # Find all th elements within the thead
    th_elements = thead.find_elements(By.TAG_NAME, 'th')

    # Initialize an empty array to store the table header content
    table_header = []

    # Iterate through each th element and extract its text content
    for th in th_elements[1:]:
        table_header.append(th.text.strip())

    # Find the table body (tbody) element
    tbody = table_section.find_element(By.TAG_NAME, 'tbody')

    # Find stripe rows
    tr_stripe_elements = tbody.find_elements(By.CLASS_NAME, 'stripe')
    tr_strong_elements = tbody.find_elements(By.CLASS_NAME, 'strong')

    # Initialize an empty array to store the Revenue data
    Revenue = []
    Net_Profit = []

    # Iterate through stripe rows to find Revenue
    for tr_stripe in tr_stripe_elements:
        td_elements = tr_stripe.find_elements(By.TAG_NAME, 'td')
        td_head = td_elements[0]
        if td_head.text.strip() == "Revenue":
            for td in td_elements[1:]:
                Revenue.append(td.text.strip())
    
    # Iterate through strong rows to find Net Profit
    for tr_strong in tr_strong_elements:
        try:
            button_element = tr_strong.find_element(By.TAG_NAME, 'button')
            if "Net Profit" in button_element.text:
                td_elements = tr_strong.find_elements(By.TAG_NAME, 'td')
                for td in td_elements[1:]:
                    Net_Profit.append(td.text.strip())
        except NoSuchElementException:
            pass
    return {"Headers" : table_header, "Revenue" : Revenue, "Net Profit" : Net_Profit}

stock_logos = ["HDFCBANK", "ICICIBANK", "KOTAKBANK", "SBIN", "BAJFINANCE", "ITC", "HINDUNILVR", "TATAMOTORS", "BAJAJ-AUTO", 
               "MARUTI", "MRF", "INFY", "TCS", "WIPRO", "SUNPHARMA", "CIPLA", "RELIANCE", "LT", "ASIANPAINT", "TITAN"]

data = {}

for stock_logo in stock_logos:
    stock_link = generate_stock_link(stock_logo)

    # Delay for 2 seconds before navigating
    time.sleep(2)

    try:
        # Navigate to the screener.com page for the specified stock
        driver.get(stock_link)
    except NoSuchWindowException as e:
        print("Error: No window available. Make sure the browser window is open.")

    time.sleep(5)

    data[stock_logo] = {
        "Quarters": get_revenue_and_net_profit_details(driver, "quarters"),
        "Annual": get_revenue_and_net_profit_details(driver, "profit-loss"),
        "ProfitAndLoss": get_profit_and_loss_details(driver)
    }

# Save the data to a JSON file
with open('stock_data.json', 'w') as json_file:
    json.dump(data, json_file)

print("Data saved to 'stock_data.json'")

# Close the webdriver
driver.quit()


Data saved to 'stock_data.json'
